# Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import collections
import gzip
import itertools
import json
import os
import sys

import numpy as np
import pandas as pd
import scipy
import matplotlib
import matplotlib.pyplot as plt
import mpl_toolkits.axes_grid1
import seaborn as sns

import yaml
from IPython.display import Image, display
from tqdm.auto import tqdm
import pysam

Set up directory paths
- The "input directory" refers to where your configuration files (especially `config.yaml`) are located. See the [ChIP-DIP GitHub repository](https://github.com/GuttmanLab/chipdip-pipeline) for more information.
- The provided paths below assume that (1) this notebook is launched from the "input directory," and (2) the "working directory" for the pipeline (where Snakemake was launched) is the same as the "input directory."

In [ ]:
# Modify these paths as necessary/desired
DIR_INPUT = os.getcwd()
DIR_ANALYSIS = os.path.join(DIR_INPUT, 'analysis') # for analysis files, such as CSV and NumPy .npy files
DIR_FIGURES = os.path.join(DIR_ANALYSIS, 'figures') # for figures
os.makedirs(DIR_FIGURES, exist_ok=True)

Load sample and pipeline configuration information

In [ ]:
with open(os.path.join(DIR_INPUT, 'config', 'config.yaml')) as f:
    config = yaml.safe_load(f)

if os.path.isabs(config['samples']):
    path_samples = config['samples']
else:
    path_samples = os.path.join(DIR_INPUT, config['samples'])
with open(path_samples, 'r') as f:
    samples = tuple(json.load(f).keys())

if os.path.isabs(config['output_dir']):
    DIR_WORKUP = config['output_dir']
else:
    DIR_WORKUP = os.path.join(DIR_INPUT, config['output_dir'])

max_chromatin_cluster_size = config.get('max_size', 10000)
min_oligos = config.get('min_oligos', 2)
proportion = config.get('proportion', 0.8)

In [ ]:
# add pipeline scripts directory to path to be able to import custom modules
sys.path.append(os.path.join(config['scripts_dir'], 'python'))
from clusters_to_interaction import label_counts_to_interaction_chunked

Number of processes to use for parallel computation

In [ ]:
n_processes = 5

## Analysis output files

In [ ]:
csv_out = lambda key: os.path.join(DIR_ANALYSIS, key + '.csv')
figs_out = lambda key: os.path.join(DIR_FIGURES, key + '.png')
paths_out = {
    'label_distribution': os.path.join(DIR_ANALYSIS, 'label_distribution.json.gz'),
}

# Sequencing QC and Pipeline Metrics

Read counts for entire sequencing run

In [ ]:
pipeline_metrics = pd.read_csv(os.path.join(DIR_WORKUP, 'qc', 'pipeline_counts.csv'), index_col=0)
pipeline_metrics['name'] = pipeline_metrics['depth'].map(lambda x: '  ' * max(x, 0) + '- ' * min(x, 1)) + pipeline_metrics['level']
pipeline_metrics[['sample', 'name', 'n_reads', 'frac_parent', 'frac_base', 'frac_root']] \
    .style.set_properties(**{
        'text-align': 'left',
        'font-family': 'Consolas',
        'white-space': 'pre'})

In [ ]:
read_counts = pipeline_metrics.set_index(['sample', 'level'])['n_reads'].astype(int)

## PCR Duplicates

Estimate library complexity based on Poisson model [[notes]](https://github.com/bentyeh/resources/blob/main/bioinformatics/models_genomics.md#general-library-preparation-and-sequencing-considerations)

In [ ]:
df_duplication = read_counts.loc[(slice(None), ['extract_barcode_to_tags', 'merge_dpm', 'bpm_fastq_to_bam', 'merge_bpm'])].to_frame()
df_duplication['description'] = df_duplication.index.get_level_values(1).map({
    'extract_barcode_to_tags': 'Number of filtered chromatin reads',
    'merge_dpm': 'Number of filtered deduplicated chromatin reads',
    'bpm_fastq_to_bam': 'Number of oligo reads',
    'merge_bpm': 'Number of deduplicated oligo reads',
})
df_duplication = df_duplication.reset_index().rename(columns={'n_reads': 'value'}).drop(columns='level')
df_duplication['molecule type'] = df_duplication['description'].str.extract('(oligo|chromatin)')
df_duplication['deduplication'] = df_duplication['description'].map(lambda s: 'deduplicated' if 'deduplicated' in s else 'none')
df_duplication = pd.concat(
    (
        df_duplication,
        (
            df_duplication
            .groupby(['sample', 'molecule type'])[['deduplication', 'value']] \
            .apply(
                lambda group:
                    1 - group.loc[group['deduplication'] == 'deduplicated', 'value'].item() /
                        group.loc[group['deduplication'] == 'none', 'value'].item()
            )
            .rename('value')
            .to_frame()
            .reset_index()
            .pipe(lambda df: df.assign(description='Proportion of ' + df['molecule type'] + ' reads that are duplicates'))
        )
    ),
    axis=0
)

In [ ]:
results = []
for sample in list(samples) + ['all']:
    count_chromatin_reads = read_counts.loc[(sample, 'extract_barcode_to_tags')]
    count_chromatin_dedup = read_counts.loc[(sample, 'merge_dpm')]
    res_chromatin = scipy.optimize.minimize_scalar(
        fun=lambda M: (M * (1 - np.exp(-count_chromatin_reads/M)) - count_chromatin_dedup)**2,
        bracket=(count_chromatin_dedup, count_chromatin_reads))
    assert res_chromatin.success and res_chromatin.fun < 1e-3 and res_chromatin.x > count_chromatin_dedup
    results.append(dict(sample=sample, value=res_chromatin.x, description='Estimated chromatin library complexity'))

    count_oligo_reads = read_counts.loc[(sample, 'bpm_fastq_to_bam')]
    count_oligo_dedup = read_counts.loc[(sample, 'merge_bpm')]
    res_oligo = scipy.optimize.minimize_scalar(
        fun=lambda M: (M * (1 - np.exp(-count_oligo_reads/M)) - count_oligo_dedup)**2,
        bracket=(count_oligo_dedup, count_oligo_reads))
    assert res_oligo.success and res_oligo.fun < 1e-3 and res_oligo.x > count_oligo_dedup
    results.append(dict(sample=sample, value=res_oligo.x, description='Estimated oligo library complexity'))

In [ ]:
df_duplication = pd.concat((df_duplication, pd.DataFrame(results)), axis=0) \
    .pivot(index='description', columns='sample', values='value')
display(df_duplication.loc[[
    'Number of filtered chromatin reads',
    'Number of filtered deduplicated chromatin reads',
    'Proportion of chromatin reads that are duplicates',
    'Estimated chromatin library complexity'
]].style.format('{:g}'))
display(df_duplication.loc[[
    'Number of oligo reads',
    'Number of deduplicated oligo reads',
    'Proportion of oligo reads that are duplicates',
    'Estimated oligo library complexity'
]].style.format('{:g}'))

## Split-BAM Read Counts

In [ ]:
if not os.path.exists(csv_out('splitbams_counts')):
    regex_splitbam_path = r'^(?:(?:(?P<sample>[^.]+)\.)?(?P<target>[^.]+))\.bam$'
    df_splitbams = pd.read_csv(
        os.path.join(DIR_WORKUP, 'splitbams', 'splitbam_counts.txt'),
        sep='\t',
        names=['path', 'chromatin count']
    )
    df_splitbams = pd.concat(
        (
            df_splitbams,
            df_splitbams['path'].map(lambda s: os.path.basename(s)).str.extract(regex_splitbam_path, expand=True)
        ),
        axis=1
    ).drop(columns='path')
    df_splitbams['sample'] = df_splitbams['sample'].fillna('all')
    df_splitbams = (
        df_splitbams
        .groupby('sample', group_keys=False)[df_splitbams.columns]
        .apply(lambda df: df.assign(**{'proportion of sample chromatin count': df['chromatin count'] / df['chromatin count'].sum()}))
        .pivot_table(index='target', columns='sample')
    )
    df_splitbams.to_csv(csv_out('splitbams_counts'))
    display(df_splitbams.sort_values(('proportion of sample chromatin count', 'all'), ascending=False))
else:
    df_splitbams = pd.read_csv(csv_out('splitbams_counts'), index_col=0, header=[0, 1])
    display(df_splitbams)

Proportion of deduplicated chromatin reads resolved to targets

In [ ]:
pd.concat(
    (
        df_splitbams.loc[~df_splitbams.index.isin(['uncertain', 'none', 'ambiguous', 'filtered'])] \
            .sum(axis=0).rename('resolved to targets'),
        df_splitbams.sum(axis=0).rename('deduplicated chromatin reads')
    ),
    axis=1
).T.astype({col: int for col in df_splitbams.columns[df_splitbams.columns.get_level_values(0) == 'chromatin count']})

# Overall efficiency

Proportion of all reads represented by target-resolved chromatin and deduplicated oligos

In [ ]:
pd.concat(
    (
        read_counts.loc[(slice(None), 'compress_fastq')].rename('total read count'),
        read_counts.loc[(slice(None), 'merge_bpm')].rename('deduplicated oligo count'),
        df_splitbams.loc[
            ~df_splitbams.index.isin(['uncertain', 'none', 'ambiguous', 'filtered']),
            ('chromatin count', slice(None))
        ].droplevel(0, axis=1).sum(axis=0).rename('target-resolved chromatin count')
    ),
    axis=1
).pipe(lambda df: df.assign(**{
    'overall efficiency (chromatin only)': df['target-resolved chromatin count'] / df['total read count'],
    'overall efficiency (chromatin + oligo)': (df['target-resolved chromatin count'] + df['deduplicated oligo count']) / df['total read count']
}))

# Clusters Analysis

In [ ]:
SAMPLE = 'all' # choose a sample to analyze
reprocess = True

In [ ]:
if SAMPLE == 'all':
    df_reads_per_cluster = pd.concat(
        [
            pd.read_csv(os.path.join(DIR_WORKUP, "clusters", f"{sample}.stats_reads_per_cluster.tsv.gz"), sep='\t').assign(sample=sample)
            for sample in samples
        ],
        axis=0,
        ignore_index=True
    ).groupby(['antibody ID', 'read type', 'reads per cluster'], group_keys=False)['number of clusters'].sum().reset_index()
else:
    df_reads_per_cluster = pd.read_csv(os.path.join(DIR_WORKUP, "clusters", f"{SAMPLE}.stats_reads_per_cluster.tsv.gz"), sep='\t')

In [ ]:
n_clusters = df_reads_per_cluster.loc[df_reads_per_cluster['read type'] == 'total', 'number of clusters'].sum()
count_oligo_in_clusters = df_reads_per_cluster.loc[df_reads_per_cluster['read type'] == 'BPM'].assign(n_reads=lambda tb: tb['number of clusters'] * tb['reads per cluster'])['n_reads'].sum()
count_chromatin_in_clusters = df_reads_per_cluster.loc[df_reads_per_cluster['read type'] == 'DPM'].assign(n_reads=lambda tb: tb['number of clusters'] * tb['reads per cluster'])['n_reads'].sum()

In [ ]:
pd.Series({
    'Number of clusters:': n_clusters,
    'Number of oligos:': count_oligo_in_clusters,
    'Number of chromatin:': count_chromatin_in_clusters,
    'Number of clusters without oligos':
        df_reads_per_cluster.loc[
            (df_reads_per_cluster['reads per cluster'] == 0) & (df_reads_per_cluster['read type'] == 'BPM'),
            'number of clusters'
        ].item(),
    'Number of clusters without chromatin reads':
        df_reads_per_cluster.loc[
            (df_reads_per_cluster['reads per cluster'] == 0) & (df_reads_per_cluster['read type'] == 'DPM'),
            'number of clusters'
        ].sum(),
    'Number of clusters with chromatin and oligo reads': 
        df_reads_per_cluster.loc[
            (df_reads_per_cluster['read type'] == 'DPM') & (df_reads_per_cluster['reads per cluster'] > 0) & (df_reads_per_cluster['antibody ID'] != 'none'),
            'number of clusters'
        ].sum()
})

In [ ]:
cluster_summary_template = {
    'chromatin count': 0,
    'oligo count': 0,
    'mode oligo count': 0,
    'label': None
}

def load_clusters(path_clusters: str) -> tuple[dict[str, dict[str, int]], dict[str, dict[str, int]]]:
    '''
    Arg
    - path_clusters: Path to cluster BAM file

    Returns
    - clusters_summaries: Map from barcode to cluster summary (dict of 'chromatin count', 'oligo count', 'mode oligo count', 'label')
    - label_distribution: Map from barcode to cluster oligo counts
    '''
    clusters_summaries = {}
    label_distribution = {}
    with pysam.AlignmentFile(path_clusters, 'r', check_sq=False) as f:
        for barcode, reads in itertools.groupby(
            tqdm(f.fetch(until_eof=True)),
            key=lambda read: read.get_tag('CB')
        ):
            n_DPM_reads = 0
            n_skipped = 0
            antibody_IDs_counter = collections.Counter()
            for read in reads:
                read_type = read.get_tag('RT')
                if not isinstance(read_type, str):
                    print(f"Unexpected read type: {read_type}", file=sys.stderr)
                    n_skipped += 1
                    continue
                if read_type.startswith("DPM"):
                    n_DPM_reads += 1
                elif read_type.startswith("BEAD_"):
                    antibody_IDs_counter[read_type[5:]] += 1
                else:
                    print(f"Unexpected read type: {read_type}", file=sys.stderr)
                    n_skipped += 1
            total = antibody_IDs_counter.total()
            if total == 0:
                cluster_summary = {
                    'label': None,
                    'chromatin count': n_DPM_reads,
                    'oligo count': 0,
                    'mode oligo count': 0,
                }
                label_distribution[barcode] = {}
            else:
                label_distribution[barcode] = antibody_IDs_counter
                candidate, count = antibody_IDs_counter.most_common()[0]
                if n_DPM_reads > max_chromatin_cluster_size:
                    label = "filtered"
                else:
                    if count < min_oligos:
                        label = "uncertain"
                    elif count / total < proportion:
                        label = "ambiguous"
                    else:
                        label = candidate
                cluster_summary = {
                    'label': label,
                    'chromatin count': n_DPM_reads,
                    'oligo count': total,
                    'mode oligo count': count,
                }

            clusters_summaries[barcode] = cluster_summary
    return clusters_summaries, label_distribution

In [ ]:
if not os.path.exists(csv_out('clusters_summaries')) or not os.path.exists(paths_out['label_distribution']) or reprocess:
    path_clusters = os.path.join(DIR_WORKUP, 'clusters', f'{SAMPLE}.labeled.bam' if SAMPLE != 'all' else f'all.bam')
    clusters_summaries, label_distribution = load_clusters(path_clusters)
    df_clusters = pd.DataFrame.from_dict(clusters_summaries, orient='index')
    df_clusters.to_csv(csv_out('clusters_summaries'))
    with gzip.open(paths_out['label_distribution'], 'wt') as f:
        json.dump(label_distribution, f)
else:
    df_clusters = pd.read_csv(csv_out('clusters_summaries'), index_col=0)
    with gzip.open(paths_out['label_distribution'], 'rt') as f:
        label_distribution = json.load(f)
df_clusters['mode oligo proportion'] = (df_clusters['mode oligo count'] / df_clusters['oligo count']).fillna(0)
df_clusters_with_oligo = df_clusters.loc[df_clusters['oligo count'] > 0]

In [ ]:
assert count_chromatin_in_clusters == df_clusters['chromatin count'].sum()
assert count_oligo_in_clusters == df_clusters['oligo count'].sum()
assert n_clusters == len(df_clusters)

In [ ]:
df_clusters_long = df_clusters \
    .reset_index(names='barcode') \
    .melt(id_vars=['barcode', 'label'], value_vars=['chromatin count', 'oligo count', 'mode oligo count'], value_name='count')

## Counts per target

In [ ]:
if not os.path.exists(csv_out('counts_per_target')) or reprocess:
    df_counts_per_target = (
        df_reads_per_cluster
        .groupby('antibody ID')[df_reads_per_cluster.columns]
        .apply(
            lambda g: pd.Series({
                'cluster count': g.loc[g['read type'] == 'total', 'number of clusters'].sum(),
                'chromatin count': g.loc[g['read type'] == 'DPM'].pipe(lambda tb: tb['number of clusters'] * tb['reads per cluster']).sum(),
                'oligo count': g.loc[g['read type'] == 'BPM'].pipe(lambda tb: tb['number of clusters'] * tb['reads per cluster']).sum(),
            })
        )
        .assign(**{
            'oligos per cluster': lambda tb: tb['oligo count'] / tb['cluster count'],
            'chromatin per cluster': lambda tb: tb['chromatin count'] / tb['cluster count'],
            'oligo well': lambda tb: [(s[0] + '0' + s[1]) if len(s) == 2 else s for s in tb.index.str.extract('([^-]+)$').squeeze().values]
        })
        .infer_objects()
        .rename_axis(index='label')
    )
    df_counts_per_target.to_csv(csv_out('counts_per_target'))
else:
    df_counts_per_target = pd.read_csv(csv_out('counts_per_target'), index_col=0)

In [ ]:
df_counts_per_target.sort_values('chromatin per cluster')

Check if the resolved clusters are evenly distributed among the input beads.

In [ ]:
label_order = {label: i for i, label in enumerate(df_counts_per_target['cluster count'].sort_values().index)}
unlabeled_targets = sorted([x for x in ['ambiguous', 'none', 'uncertain', 'filtered'] if x in label_order], key=lambda s: label_order[s])
labeled_targets = [label for label in label_order if label not in unlabeled_targets]

In [ ]:
if not os.path.exists(figs_out('counts_per_cluster')) or reprocess:
    with sns.axes_style('darkgrid'):    
        g = sns.FacetGrid(
            data=df_clusters[['label', 'chromatin count', 'oligo count', 'mode oligo proportion']] \
                .pipe(lambda df: df.assign(labeled=df['label'].isin(labeled_targets))) \
                .melt(id_vars=['label', 'labeled'], var_name='metric') \
                .sort_values('label', key=lambda series: series.map(label_order)),
            col='metric',
            row='labeled',
            sharex=False,
            sharey='row',
            gridspec_kws=dict(height_ratios=(0.12, 0.88)),
            margin_titles=True
        )
        g.map_dataframe(
            sns.boxplot,
            y='label',
            x='value',
            color='C0',
            showfliers=False)
        g.set_titles(col_template="{col_name}", row_template='')
        g.set_axis_labels('', '')
        g.figure.set_size_inches(12, 7)
        for (row_val, col_val), ax in g.axes_dict.items():
            ax.yaxis.grid(True)
            if row_val == False and col_val.endswith('count'):
                ax.set_xscale('symlog')
        g.figure.text(
            0.1,
            0,
            'Note: The x-axis is symlog-scaled for "chromatin count" and "oligo count" in the top row.',
            horizontalalignment='left')
        g.savefig(figs_out('counts_per_cluster'))
else:
    display(Image(filename=figs_out('counts_per_cluster')))

In [ ]:
if not os.path.exists(figs_out('counts_per_target')) or reprocess:
    df_plot_counts_per_target = df_counts_per_target \
        .assign(labeled=df_counts_per_target.index.isin(labeled_targets)) \
        .reset_index() \
        .sort_values('cluster count') \
        .drop(columns=['oligo well']) \
        .melt(id_vars=['label', 'labeled'], var_name='metric')

    with sns.axes_style('darkgrid'):
        g = sns.FacetGrid(
            data=df_plot_counts_per_target,
            col='metric',
            row='labeled',
            sharex=False,
            sharey='row',
            gridspec_kws=dict(height_ratios=(0.12, 0.88)),
            margin_titles=True
        )
        g.map_dataframe(sns.barplot, y='label', x='value', color='C0')
        g.set_titles(col_template="{col_name}", row_template='')
        g.set_axis_labels('', '')
        g.figure.set_size_inches(15, 7)
        for (row_val, col_val), ax in g.axes_dict.items():
            if row_val == False:
                ax.set_xscale('log')
        [ax.yaxis.grid(True) for ax in g.axes.flatten()]
        g.figure.text(
            0.025,
            0,
            'Note: The x-axis is log-scaled for the top row of unlabeled clusters.')
        g.tight_layout()
        g.savefig(figs_out('counts_per_target'))
else:
    display(Image(filename=figs_out('counts_per_target')))

In [ ]:
df_counts_per_target.describe()

Note that the "number of oligo reads" above (in the summary statistics and in the plots) are not the number of oligo reads matching the assigned label, but the total number of oligo reads in the cluster.

In [ ]:
df_counts_per_target[['cluster count', 'chromatin count', 'oligo count']] \
    .rename(columns=lambda s: s.replace(' count', '')) \
    .melt(value_name='count', ignore_index=False) \
    .pipe(lambda df: df.assign(proportion=df['count'] / df.groupby('variable')['count'].transform('sum'))) \
    .pivot(columns='variable')

## Cluster distributions: molecules per cluster

Distributions of molecule counts per cluster, facetted by target (label).

In [ ]:
if not all(map(os.path.exists, (figs_out('molecules_per_cluster_unlabeled'), figs_out('molecules_per_cluster_unlabeled_zoom')))) or reprocess:
    g = sns.FacetGrid(
        data=pd.concat(
            (df_clusters_long.loc[df_clusters_long['label'].isin(unlabeled_targets)],
             df_clusters_long.loc[df_clusters_long['label'].isin(labeled_targets)].assign(label='labeled')),
            axis=0,
            ignore_index=True),
        hue='variable',
        col='label',
        height=2.5,
        col_order=['labeled'] + unlabeled_targets,
        sharex=False
    )
    g.figure.set_constrained_layout(True)
    g.figure.suptitle('Distribution of molecule counts per unlabeled cluster')
    g.map(sns.ecdfplot, 'count')
    g.add_legend()
    g.savefig(figs_out('molecules_per_cluster_unlabeled'))
    display(g.figure)

    zoom_count_max = 15
    [ax.set(xlim=(-0.5, zoom_count_max + 0.5)) for ax in g.axes.flatten()]
    g.figure.suptitle('Distribution of molecule counts per unlabeled cluster (zoom)')
    g.savefig(figs_out('molecules_per_cluster_unlabeled_zoom'))
    g.figure.show()
else:
    display(Image(filename=figs_out('molecules_per_cluster_unlabeled')))
    display(Image(filename=figs_out('molecules_per_cluster_unlabeled_zoom')))

In [ ]:
if not os.path.exists(figs_out('molecules_per_cluster_labeled')) or not os.path.exists(figs_out('molecules_per_cluster_labeled_zoom')) or reprocess:
    g = sns.FacetGrid(
        data=df_clusters_long.loc[df_clusters_long['label'].isin(labeled_targets)],
        hue='variable',
        col='label',
        height=2.5,
        col_wrap=4,
        col_order=labeled_targets,
        sharex=False
    )
    g.figure.set_constrained_layout(True)
    g.figure.suptitle('Distribution of molecule counts per labeled cluster', y=1)
    g.map(sns.ecdfplot, 'count')
    g.add_legend()
    g.savefig(figs_out('molecules_per_cluster_labeled'))
    display(g.figure)

    zoom_count_max = 15
    [ax.set(xlim=(-0.5, zoom_count_max + 0.5)) for ax in g.axes.flatten()]
    g.figure.suptitle('Distribution of molecule counts per labeled cluster (zoom)', y=1)
    g.savefig(figs_out('molecules_per_cluster_labeled_zoom'))
    g.figure.show()
else:
    display(Image(filename=figs_out('molecules_per_cluster_labeled')))
    display(Image(filename=figs_out('molecules_per_cluster_labeled_zoom')))

Oligos per cluster

In [ ]:
if not os.path.exists(figs_out('oligos_per_cluster')) or reprocess:
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 5), constrained_layout=True, sharex='row', sharey='col')
    fig.suptitle('Distribution of oligos per cluster')

    axs[0, 0].hist(df_clusters['oligo count'], bins=50, log=True, align='left')
    axs[0, 0].set(xlabel='Oligos per cluster', ylabel='Number of clusters', title='Histogram')
    sns.ecdfplot(data=df_clusters, x='oligo count', ax=axs[0, 1])
    axs[0, 1].set(xlabel='Oligos per cluster', ylabel='Proportion of clusters', title='eCDF')

    # zoom
    zoom_oligo_max = 70
    mask_oligo_counts_zoom = df_clusters['oligo count'] <= zoom_oligo_max
    bins = np.arange(zoom_oligo_max + 2) - 0.5
    axs[1, 0].hist(df_clusters.loc[mask_oligo_counts_zoom, 'oligo count'], bins=bins, log=True, align='mid')
    axs[1, 0].set(xticks=np.arange(0, zoom_oligo_max + 2, 3), xlabel='Oligos per cluster', ylabel='Number of clusters', title='Histogram (zoom)')
    sns.ecdfplot(data=df_clusters, x='oligo count', ax=axs[1, 1])
    axs[1, 1].set(xlim=(-0.5, zoom_oligo_max + 0.5), xlabel='Oligos per cluster', ylabel='Proportion of clusters', title='eCDF (zoom)')
    fig.savefig(figs_out('oligos_per_cluster'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('oligos_per_cluster')))

Mode oligo proportion

In [ ]:
if not os.path.exists(figs_out('mode_oligo_proportion')) or reprocess:
    fig, axs = plt.subplots(nrows=3, ncols=2, figsize=(10, 8), constrained_layout=True, sharex=True, sharey='col')
    fig.suptitle('Distribution of mode oligo proportion')

    axs[0, 0].hist(df_clusters['mode oligo proportion'], log=True, bins=50)
    axs[0, 0].set(ylabel='Number of clusters')
    sns.ecdfplot(df_clusters['mode oligo proportion'], ax=axs[0, 1])
    axs[0, 1].set(ylabel='Proportion of clusters')

    axs[1, 0].hist(df_clusters_with_oligo['mode oligo proportion'], log=True, bins=50)
    axs[1, 0].set(
        ylabel='Number of clusters',
        title='Filtered for > 0 oligos per cluster')
    sns.ecdfplot(df_clusters_with_oligo['mode oligo proportion'], ax=axs[1, 1])
    axs[1, 1].set(
        ylabel='Proportion of clusters',
        title='Filtered for > 0 oligos per cluster')

    mask_oligos_gt1 = df_clusters_with_oligo['oligo count'] > 1
    axs[2, 0].hist(df_clusters_with_oligo.loc[mask_oligos_gt1, 'mode oligo proportion'], log=True, bins=50)
    axs[2, 0].set(
        xlabel='Mode oligo proportion',
        ylabel='Number of clusters',
        title='Filtered for > 1 oligo per cluster')
    sns.ecdfplot(df_clusters_with_oligo.loc[mask_oligos_gt1, 'mode oligo proportion'], ax=axs[2, 1])
    axs[2, 1].set(
        xlabel='Mode oligo proportion',
        ylabel='Proportion of clusters',
        title='Filtered for > 1 oligo per cluster')
    fig.savefig(figs_out('mode_oligo_proportion'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('mode_oligo_proportion')))

In [ ]:
('Proportion of clusters with mixed oligos: '
 f"{(df_clusters_with_oligo['mode oligo proportion'] < 1).sum() / n_clusters:.4f}")

In [ ]:
('Proportion of clusters with oligo mixing > 20%: '
 f"{(df_clusters_with_oligo['mode oligo proportion'] < 0.8).sum() / n_clusters:.4f}")

Chromatin per cluster

In [ ]:
df_clusters['chromatin count'].describe()

In [ ]:
if not os.path.exists(figs_out('chromatin_per_cluster')) or reprocess:
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 5), constrained_layout=True, sharex='row', sharey='col')
    fig.suptitle('Distribution of chromatin per cluster')

    axs[0, 0].hist(df_clusters['chromatin count'], bins=50, log=True, align='left')
    axs[0, 0].set(xlabel='Chromatin per cluster', ylabel='Number of clusters', title='Histogram')
    sns.ecdfplot(data=df_clusters, x='chromatin count', ax=axs[0, 1])
    axs[0, 1].set(xlabel='Chromatin per cluster', ylabel='Proportion of clusters', title='eCDF')
    axs[0, 0].axvline(x=max_chromatin_cluster_size, c='C1', label='chromatin filter threshold')
    axs[0, 1].axvline(x=max_chromatin_cluster_size, c='C1', label='chromatin filter threshold')
    axs[0, 1].legend()

    # zoom
    zoom_chromatin_max = 15
    mask_chromatin_counts_zoom = df_clusters['chromatin count'] <= zoom_chromatin_max
    bins = np.arange(zoom_chromatin_max + 2) - 0.5
    axs[1, 0].hist(df_clusters.loc[mask_chromatin_counts_zoom, 'chromatin count'], bins=bins, log=True, align='mid')
    axs[1, 0].set(
        xticks=np.arange(zoom_chromatin_max + 2),
        xlabel='Chromatin per cluster',
        ylabel='Number of clusters',
        title='Histogram (zoom)')
    sns.ecdfplot(data=df_clusters, x='chromatin count', ax=axs[1, 1])
    axs[1, 1].set(
        xlim=(-0.5, zoom_chromatin_max + 0.5),
        xlabel='Chromatin per cluster',
        ylabel='Proportion of clusters',
        title='eCDF (zoom)')

    fig.savefig(figs_out('chromatin_per_cluster'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('chromatin_per_cluster')))

## Chromatin distributions

Chromatin vs. chromatin per cluster

In [ ]:
if not os.path.exists(figs_out('chromatin_v_chromatin_per_cluster')) or reprocess:
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 5), constrained_layout=True, sharex='row', sharey='col')
    fig.suptitle('Distribution of chromatin per cluster')

    axs[0, 0].hist(df_clusters['chromatin count'], weights=df_clusters['chromatin count'], bins=50, log=True, align='left')
    axs[0, 0].set(xlabel='Chromatin per cluster', ylabel='Number of chromatin', title='Histogram')
    sns.ecdfplot(data=df_clusters, x='chromatin count', weights='chromatin count', ax=axs[0, 1])
    axs[0, 1].set(xlabel='Chromatin per cluster', ylabel='Proportion of chromatin', title='eCDF')
    axs[0, 0].axvline(x=max_chromatin_cluster_size, c='C1', label='chromatin filter threshold')
    axs[0, 1].axvline(x=max_chromatin_cluster_size, c='C1', label='chromatin filter threshold')
    axs[0, 1].legend()

    # zoom
    zoom_chromatin_max = 15
    mask_chromatin_counts_zoom = df_clusters['chromatin count'] <= zoom_chromatin_max
    bins = np.arange(zoom_chromatin_max + 2) - 0.5
    axs[1, 0].hist(df_clusters.loc[mask_chromatin_counts_zoom, 'chromatin count'],
                weights=df_clusters.loc[mask_chromatin_counts_zoom, 'chromatin count'],
                bins=bins, log=True, align='mid')
    axs[1, 0].set(
        xticks=np.arange(zoom_chromatin_max + 2),
        xlabel='Chromatin per cluster',
        ylabel='Number of chromatin',
        title='Histogram (zoom)')
    sns.ecdfplot(data=df_clusters, x='chromatin count', weights='chromatin count', ax=axs[1, 1])
    axs[1, 1].set(
        xlim=(-0.5, zoom_chromatin_max + 0.5),
        xlabel='Chromatin per cluster',
        ylabel='Proportion of chromatin',
        title='eCDF (zoom)')

    fig.savefig(figs_out('chromatin_v_chromatin_per_cluster'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('chromatin_v_chromatin_per_cluster')))

Chromatin vs. oligos per cluster

In [ ]:
if not os.path.exists(figs_out('chromatin_per_oligo_per_cluster')) or reprocess:
    fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10, 5), constrained_layout=True, sharex='row', sharey='col')
    fig.suptitle('Distribution of chromatin per oligo per cluster')

    sns.histplot(df_clusters, x='oligo count', weights='chromatin count', bins=50, log=True, ax=axs[0, 0])
    axs[0, 0].set(xlabel='Oligos per cluster', ylabel='Number of chromatin', title='Histogram')
    sns.ecdfplot(df_clusters, x='oligo count', weights='chromatin count', ax=axs[0, 1])
    axs[0, 1].set(xlabel='Oligos per cluster', ylabel='Proportion of chromatin', title='eCDF')

    # zoom
    zoom_oligo_max = 15
    mask_oligo_counts_zoom = df_clusters['oligo count'] <= zoom_oligo_max
    sns.histplot(
        df_clusters.loc[mask_oligo_counts_zoom],
        x='oligo count',
        weights='chromatin count',
        bins=zoom_oligo_max + 1,
        ax=axs[1, 0])
    axs[1, 0].set(xlabel='Oligos per cluster', ylabel='Number of chromatin', title='Histogram (zoom)')
    sns.ecdfplot(df_clusters, x='oligo count', weights='chromatin count', ax=axs[1, 1])
    axs[1, 1].set(
        xlabel='Oligos per cluster',
        ylabel='Proportion of chromatin',
        title='eCDF (zoom)',
        xlim=(0, zoom_oligo_max))

    fig.savefig(figs_out('chromatin_per_oligo_per_cluster'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('chromatin_per_oligo_per_cluster')))

Chromatin vs. mode oligo proportion

In [ ]:
if not os.path.exists(figs_out('chromatin_v_mode_oligo_proportion')) or reprocess:
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 3), constrained_layout=True, sharex=True)
    fig.suptitle('Distribution of chromatin per mode oligo proportion')

    axs[0].hist(df_clusters['mode oligo proportion'], weights=df_clusters['chromatin count'], bins=50, log=True)
    # much slower: sns.histplot(df_clusters, x='mode oligo proportion', weights='chromatin count', bins=50, log=True)
    axs[0].set(xlabel='Mode oligo proportion', ylabel='Number of chromatin', xlim=(0, 1))

    chromatin_per_mode_oligo_proportion = df_clusters_with_oligo.groupby('mode oligo proportion')['chromatin count'].sum()
    bins = chromatin_per_mode_oligo_proportion.index.values
    bins = np.append(bins, bins.max() + 0.01)
    axs[1].stairs(
        (chromatin_per_mode_oligo_proportion / chromatin_per_mode_oligo_proportion.sum()).cumsum(),
        bins)
    # much slower: sns.ecdfplot(df_clusters, x='mode oligo proportion', weights='chromatin count', ax=axs[1])
    axs[1].set(xlabel='Mode oligo proportion', ylabel='Proportion of chromatin', title='eCDF', ylim=(0, 1))

    fig.savefig(figs_out('chromatin_v_mode_oligo_proportion'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('chromatin_v_mode_oligo_proportion')))

In [ ]:
('Chromatin in clusters with mixed oligos: '
 f"{df_clusters_with_oligo.loc[df_clusters_with_oligo['mode oligo proportion'] < 1, 'chromatin count'].sum()} "
 f"({df_clusters_with_oligo.loc[df_clusters_with_oligo['mode oligo proportion'] < 1, 'chromatin count'].sum() / count_chromatin_in_clusters:.2%})")

In [ ]:
(f'Chromatin in clusters with mode oligo propotion < {proportion} and 1+ oligo: '
 f"{df_clusters_with_oligo.loc[df_clusters_with_oligo['mode oligo proportion'] < proportion, 'chromatin count'].sum()} "
 f"({df_clusters_with_oligo.loc[df_clusters_with_oligo['mode oligo proportion'] < proportion, 'chromatin count'].sum() / count_chromatin_in_clusters:.2%})")

In [ ]:
(f'Chromatin in clusters with mode oligo propotion < {proportion}, including clusters without oligos: '
 f"{df_clusters.loc[df_clusters['mode oligo proportion'] < 0.8, 'chromatin count'].sum()} "
 f"({df_clusters.loc[df_clusters['mode oligo proportion'] < 0.8, 'chromatin count'].sum() / count_chromatin_in_clusters:.2%})")

## Joint distributions of oligo and chromatin per cluster

Heatmap of oligos per cluster vs. mode oligo proportion

In [ ]:
if not os.path.exists(figs_out('oligo_count_v_mode_oligo_proportion')) or reprocess:
    max_oligo_counts_thresh = 15
    cols = ['mode oligo proportion', 'oligo count']
    mask_oligo_counts_lte_thresh = df_clusters_with_oligo['oligo count'] <= max_oligo_counts_thresh
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 3), constrained_layout=True, sharex=True, sharey=True)
    sns.histplot(
        data=df_clusters_with_oligo.loc[mask_oligo_counts_lte_thresh, cols],
        x=cols[0], y=cols[1],
        binwidth=(0.1, 1),
        cbar=True, cbar_kws=dict(label='number of clusters'),
        ax=axs[0])
    axs[0].set(title='Distribution of clusters')

    mask_oligos_gt1 = df_clusters_with_oligo['oligo count'] > 1
    sns.histplot(
        data=df_clusters_with_oligo.loc[mask_oligo_counts_lte_thresh & mask_oligos_gt1, cols],
        x=cols[0], y=cols[1],
        binwidth=(0.1, 1),
        cbar=True, cbar_kws=dict(label='number of clusters'),
        ax=axs[1])
    axs[1].set(title='Distribution of clusters with > 1 oligo')

    sns.histplot(
        data=df_clusters_with_oligo.loc[mask_oligo_counts_lte_thresh, cols + ['chromatin count']],
        x=cols[0], y=cols[1], weights='chromatin count',
        binwidth=(0.1, 1),
        cbar=True, cbar_kws=dict(label='chromatin count'),
        ax=axs[2])
    axs[2].set(
        xlim=(0, 1),
        ylim=(1, max_oligo_counts_thresh),
        title='Distribution of chromatin')
    fig.savefig(figs_out('oligo_count_v_mode_oligo_proportion'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('oligo_count_v_mode_oligo_proportion')))

## Analysis of unlabeled clusters

### Uncertain clusters

Mode oligo count = 1

What is the distribution of oligos (labels)?
- In how many uncertain clusters is each oligo found in?
- How much chromatin is contained in those clusters?

In [ ]:
if not os.path.exists(csv_out('uncertain_counts_per_target')) or reprocess:
    uncertain_target_template = {
        'unique cluster': 0,
        'unique chromatin': 0,
        'mixed cluster': 0,
        'mixed chromatin': 0
    }
    uncertain_target_distribution = {target: uncertain_target_template.copy() for target in labeled_targets}
    for barcode, row in tqdm(df_clusters.loc[df_clusters['label'] == 'uncertain'].iterrows(), total=df_counts_per_target.loc['uncertain', 'cluster count'].item()):
        cluster_type = 'mixed' if row['oligo count'] > 1 else 'unique'
        for target in label_distribution[barcode]:
            uncertain_target_distribution[target][f'{cluster_type} cluster'] += 1
            uncertain_target_distribution[target][f'{cluster_type} chromatin'] += row['chromatin count']
    df_uncertain = pd.DataFrame.from_dict(uncertain_target_distribution, orient='index')
    df_uncertain.to_csv(csv_out('uncertain_counts_per_target'))
else:
    df_uncertain = pd.read_csv(csv_out('uncertain_counts_per_target'), index_col=0)

In [ ]:
if not os.path.exists(figs_out('uncertain_target_distribution')) or reprocess:
    with sns.axes_style('darkgrid'):
        g = sns.catplot(
            data=df_uncertain \
                .reset_index(names='label') \
                .melt(id_vars='label', value_name='count') \
                .pipe(lambda df: pd.concat(
                    (df[['label', 'count']],
                    df['variable'].str.extract('(?P<cluster_type>unique|mixed) (?P<unit>cluster|chromatin)')),
                    axis=1)) \
                .pipe(lambda df: df.assign(cluster_type=df['cluster_type'].map({'unique': 'unique oligo', 'mixed': 'mixed oligos'}))) \
                .sort_values('label', key=lambda series: series.map(label_order)),
            kind='bar',
            x='count',
            y='label',
            hue='cluster_type',
            col='unit',
            col_order=('cluster', 'chromatin'),
            sharex=False
        )
        [ax.yaxis.grid(True) for ax in g.axes.flatten()]
        g.figure.suptitle('Distribution of oligos in uncertain clusters', y=1)
        g.figure.set_size_inches(8, 7)
        sns.move_legend(g, 'right', bbox_to_anchor=(1.05, 0.5))
        g.savefig(figs_out('uncertain_target_distribution'))
else:
    display(Image(filename=figs_out('uncertain_target_distribution')))

In [ ]:
if not os.path.exists(figs_out('uncertain_label_representation')) or reprocess:
    with sns.axes_style('darkgrid'):
        df_uncertain_compare = pd.concat(
            (df_uncertain[['unique cluster', 'unique chromatin']],
            df_counts_per_target[['cluster count', 'chromatin count']]),
            axis=1) \
            .dropna(axis=0) \
            .apply(lambda s: s/s.sum(), axis=0)
        df_uncertain_compare['cluster proportion diff'] = df_uncertain_compare['unique cluster'] - df_uncertain_compare['cluster count']
        df_uncertain_compare['chromatin proportion diff'] = df_uncertain_compare['unique chromatin'] - df_uncertain_compare['chromatin count']

        fig, ax = plt.subplots(figsize=(6, 7), constrained_layout=True)
        fig.suptitle('Over/underrepresentation of labels among uncertain clusters')
        sns.barplot(
            data=df_uncertain_compare[['cluster proportion diff', 'chromatin proportion diff']] \
                .rename(columns={'cluster proportion diff': 'cluster', 'chromatin proportion diff': 'chromatin'})
                .reset_index(names='label') \
                .melt(id_vars='label', var_name='unit', value_name='proportion difference') \
                .sort_values('label', key=lambda series: series.map(label_order)),
            x='proportion difference',
            y='label',
            hue='unit',
            ax=ax
        )
        fig.text(
            0.1,
            -0.05,
            ('Proportion difference =\n'
             '    fraction of uncertain clusters/chromatin with a given label\n'
             '    - fraction of properly labeled clusters/chromatin with a given label'),
             horizontalalignment='left')
        g.savefig(figs_out('uncertain_label_representation'))
else:
    display(Image(filename=figs_out('uncertain_label_representation')))

Interpretation
- Most targets comprise a similar fraction of chromatin/clusters that are labeled by a single oligo as by multiple oligos.
- A larger-than-expected fraction of chromatin in clusters with a single oligo are labeled by (targets with positive proportion difference values) compared to chromatin in properly labeled clusters.
- A smaller-than-expected fraction of chromatin in clusters with a single oligo are labeled by (targets with negative proportion difference values) compared to chromatin in properly labeled clusters.

### Ambiguous clusters

- Mode oligo count >= 2 reads
- Mode oligo proportion < 0.8
- Chromatin count <= 10000

Interpretation
- Oligo hopping --> normal oligo count
  - Check if chromatin count matches distribution of most common oligo 
- Multiple protein G-antibody beads stuck together throughout split-pool barcoding --> large oligo count

In [ ]:
if not os.path.exists(figs_out('ambiguous_cluster_oligo_counts')) or reprocess:
    fig, axs = plt.subplots(ncols=2, figsize=(8, 3), constrained_layout=True, sharex=True)
    fig.suptitle('Distribution of oligo count among clusters')
    axs[0].hist(
        df_clusters.loc[df_clusters['label'].isin(labeled_targets), 'oligo count'].pipe(lambda s: s.loc[(s < 20) & (s > 2)]),
        bins=np.arange(3, 21),
        log=True,
        align='left'
    )
    axs[0].set(xlabel='oligos per cluster', ylabel='number of clusters', title='Labeled clusters')
    axs[1].hist(
        df_clusters.loc[df_clusters['label'] == 'ambiguous', 'oligo count'].pipe(lambda s: s.loc[s < 20]),
        bins=np.arange(3, 21),
        log=True,
        align='left')
    axs[1].set(
        xticks=np.arange(3, 21, 3),
        xlabel='oligos per cluster',
        ylabel='number of clusters',
        title='Ambiguous clusters'
    )
    fig.savefig(figs_out('ambiguous_cluster_oligo_counts'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('ambiguous_cluster_oligo_counts')))

There may be a large drop in the number of ambiguous clusters from 4 to 5 oligos per cluster (and similar but smaller drops from 9 to 10 and from 14 to 15 oligos per cluster), which we do not observe among properly labeled clusters. This is perhaps not unexpected, since the threshold for labeling a cluster is a proportion of 0.8 - which allows for 1 "wrong" oligo out of 5. This analysis suggests that the `proportion=0.8` threshold of labeling clusters is very conserative: most of the clusters with 4 oligos likely contain 1 "wrong" oligo and 3 "correct" oligos.

In [ ]:
df_clusters.loc[(df_clusters['oligo count'] == 4) & (df_clusters['label'] == 'ambiguous')].index \
    .to_series() \
    .map(lambda barcode: tuple(sorted(label_distribution[barcode].values()))) \
    .value_counts() \
    .rename('cluster count') \
    .to_frame() \
    .pipe(lambda df: df.assign(proportion=df['cluster count'] / df['cluster count'].sum()))

Check the amount of chromatin in clusters with 3 presumably "correct" oligos and 1 "wrong" oligo.

In [ ]:
df_clusters.loc[(df_clusters['oligo count'] == 4) & (df_clusters['label'] == 'ambiguous')] \
    .pipe(lambda df: df.loc[df.index.to_series().map(lambda barcode: tuple(sorted(label_distribution[barcode].values())) == (1, 3))]) \
    ['chromatin count'].sum()

In [ ]:
if not os.path.exists(figs_out('ambiguous_clusters_heatmap')) or reprocess:
    fig, ax = plt.subplots(constrained_layout=True)
    fig.suptitle('Distribution of ambiguous clusters by chromatin count and oligo count')
    sns.histplot(
        data=df_clusters.loc[df_clusters['label'] == 'ambiguous', ['oligo count', 'chromatin count']].map(lambda x: np.maximum(x, 0.1)),
        x='oligo count',
        y='chromatin count',
        bins=25,
        cbar=True,
        cbar_kws=dict(label='Number of clusters'),
        log_scale=(True, True),
        norm=matplotlib.colors.LogNorm(),
        vmin=None, vmax=None,
        ax=ax
    )
    ax.set_yticks(
        ticks=[1e-1, 1e0, 1e1, 1e2, 1e3, 1e4],
        labels=['$0$', '$1$', '$\\mathdefault{10^{1}}$', '$\\mathdefault{10^{2}}$', '$\\mathdefault{10^{3}}$', '$\\mathdefault{10^{4}}$']
    )
    fig.savefig(figs_out('ambiguous_clusters_heatmap'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('ambiguous_clusters_heatmap')))

### Filtered clusters

Interpretation
- A chromatin-dense cluster with high oligo count (relative to properly labeled clusters) presumably reflects multiple protein G-antibody beads stuck together throughout split-pool barcoding. This could potentially indicate colocalization of target proteins on a region of chromatin.
- A chromatin-dense cluster with low oligo count presumably reflects an aggregate of crosslinked chromatin immunoprecipitated onto an individual bead. This could potentially indicate chromatin-chromatin interactions.

Check oligo distribution of filtered clusters

In [ ]:
if not os.path.exists(figs_out('filtered_cluster_oligo_counts')) or reprocess:
    fig, axs = plt.subplots(ncols=2, figsize=(8, 3), constrained_layout=True, sharex=True)
    fig.suptitle('Distribution of oligo count among clusters')
    axs[0].hist(
        df_clusters.loc[df_clusters['label'].isin(labeled_targets), 'oligo count'], #.pipe(lambda s: s.loc[(s < 20) & (s > 2)]),
        bins=50,
        range=(1, 1000),
        log=True
    )
    axs[0].set(xlabel='oligos per cluster', ylabel='number of clusters', title='Labeled clusters')
    axs[1].hist(
        df_clusters.loc[df_clusters['label'] == 'filtered', 'oligo count'], #.pipe(lambda s: s.loc[s < 20]),
        bins=50,
        range=(1, 1000),
        log=True
    )
    axs[1].set(
        xlabel='oligos per cluster',
        ylabel='number of clusters',
        title='Filtered clusters'
    )
    fig.text(0.1, -0.05, 'Note: There are a few clusters with >1000 oligos not shown in this figure.')
    fig.savefig(figs_out('filtered_cluster_oligo_counts'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('filtered_cluster_oligo_counts')))

# Colocalization Analysis

Idea: Clusters with multiple oligos may represent colocalization of targets on chromatin.

Criteria for including a target in a cluster
- Count of target oligo >= 2
- Proportion of target oligo >= 0.1

Colocalization heatmaps
- Restrict to clusters with chromatin
- Apply criteria for including targets in a clusters
- Normalization strategy 1
  - Interaction frequency matrix $M$ construction: for each cluster containing both target $i$ and target $j \neq 1$, increment $M_{i,j}$ and $M_{j,i}$ each by 1.
    - Include self-interactions: increment $M_{i,i}$ by 1
  - Matrix normalization: divide each row by the number of clusters that the row-target is in (i.e., the diagonal element value of that row)
    - $\tilde{M}_{i,j} = M_{i,j} / M_{i,i}$
    - Normalized value interpretation: $\tilde{M}_{i,j}$ is the proportion of chromatin-containing target $i$ clusters that also contain target $j$.
- Normalization strategy 2
  - Interaction frequency matrix $M$ construction: for each cluster containing both target $i$ and target $j$, increment $M_{i,j}$ and $M_{j,i}$ each by $2/n$, where $n$ is the number of targets in that cluster.
  - Matrix normalization: divide each row by the number of pairwise interactions that the row-target is in (i.e., the row sum, excluding the diagonal element)
    - Ignore self-interactions: $\tilde{M}_{i,i} = 1$
    - Normalized value interpretation: $\tilde{M}_{i,j}$ is the proportion of target $i$'s *pairwise colocalizations* on chromatin involving target $j$.

In [ ]:
def normalize_by_clusters(df, pseudocount=None):
    '''
    Normalize a matrix of interactions by the number of clusters.
    - NA values (i.e., targets not present in any cluster) are replaced with zeros.
    - 0 values are optionally replaced with a pseudocount.

    Args
    - df: pd.DataFrame. shape=(n, n)
        df.loc[i, j] gives the count of clusters containing both targets i and j.
        df.loc[i, i] gives the number of clusters containing target i.
    - pseudocount: numeric or str. default=None
        If None, do not add a pseudocount.
        If numeric, add this value to all entries in df.
        If 'min_over_ten', replace 0s with the minimum positive value in df / 10.

    Returns: pd.DataFrame. shape=(n, n)
    '''
    df_norm = df / np.diag(df)[:, None]
    df_norm.fillna(0, inplace=True)
    if isinstance(pseudocount, (int, float)):
        df_norm += pseudocount
    elif pseudocount == 'min_over_ten':
        df_norm.replace(0, df_norm.values[df_norm.values > 0].min() / 10, inplace=True)
    else:
        assert pseudocount is None
    return df_norm

def normalize_by_pairwise(df, pseudocount=None):
    '''
    Normalize a matrix of pairwise interactions by the number of pairwise interactions.
    - NA values (i.e., targets not present in any cluster) are replaced with zeros.
    - 0 values are optionally replaced with a pseudocount.
    - The diagonal of the returned matrix is set to 1.

    Args
    - df: pd.DataFrame. shape=(n, n)
        df.loc[i, j] gives the count of pairwise containing both targets i and j.
        df.loc[i, i] is ignored
    - pseudocount: numeric or str. default=None
        If None, do not add a pseudocount.
        If numeric, add this value to all entries in df.
        If 'min_over_ten', replace 0s with the minimum positive value in df / 10.

    Returns: pd.DataFrame. shape=(n, n)
    '''
    df_norm = df.copy()
    np.fill_diagonal(df_norm.values, 0)
    df_norm = df_norm / df_norm.values.sum(axis=1, keepdims=True)
    df_norm.fillna(0, inplace=True)
    np.fill_diagonal(df_norm.values, 1)
    if isinstance(pseudocount, (int, float)):
        df_norm += pseudocount
    elif pseudocount == 'min_over_ten':
        df_norm.replace(0, df_norm.values[df_norm.values > 0].min() / 10, inplace=True)
    else:
        assert pseudocount is None
    return df_norm

## All clusters

Hierarchical clustering of barcodes (clusters) by which targets they contain.
- To reduce matrix size, restrict to clusters with putative real interactions meeting the following criteria:
  - Contain chromatin
  - Contain 2+ real targets

In [ ]:
if not os.path.exists(figs_out('clustermap')) or reprocess:
    df_interact = df_clusters.loc[
        (df_clusters['chromatin count'] > 0) &
        (df_clusters['oligo count'] > 1) &
        (df_clusters['mode oligo count'] < df_clusters['oligo count'])]
    df_interact = pd.DataFrame(
        [label_distribution[barcode] for barcode in df_interact.index],
        columns=labeled_targets
    ).fillna(0).astype(int)
    mask_multiple_real_targets = (
        ((df_interact >= 2).sum(axis=1) >= 2) &
        ((df_interact.div(df_interact.sum(axis=1), axis=0) >= 0.1).sum(axis=1) >= 2))
    g = sns.clustermap(
        df_interact.loc[mask_multiple_real_targets].pipe(lambda df: df.div(df.sum(axis=1), axis=0)).T,
        xticklabels=False,
        cbar_kws=dict(label='proportion of oligo\nin cluster'))
    g.savefig(figs_out('clustermap'))
    g.figure.show()
else:
    display(Image(filename=figs_out('clustermap')))

In [ ]:
if not all(map(
    os.path.exists,
    (csv_out('interactions_all_pairwise'),
     csv_out('interactions_all_clusters'),
     figs_out('interactions_all')))) or reprocess:

    mask_interacting_clusters = \
        (df_clusters['chromatin count'] > 0) & \
        (df_clusters['mode oligo count'] > 1) & \
        (df_clusters['mode oligo count'] >= df_clusters['oligo count'] * 0.1)

    interactions_all_clusters = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_all_clusters = pd.DataFrame(interactions_all_clusters, index=labeled_targets, columns=labeled_targets)
    # save as DataFrame to preserve index and column labels
    df_interactions_all_clusters.to_csv(csv_out('interactions_all_clusters'))

    interactions_all_pairwise = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        n_processes=n_processes,
        downweighting='n_over_two',
        dtype_in=float)
    df_interactions_all_pairwise = pd.DataFrame(interactions_all_pairwise, index=labeled_targets, columns=labeled_targets)
    df_interactions_all_pairwise.to_csv(csv_out('interactions_all_pairwise'))

    fig, axs = plt.subplots(1, 2, width_ratios=(1, 1), figsize=(10, 5), layout='constrained', gridspec_kw=dict(wspace=0.2))
    fig.suptitle('Colocalization in all chromatin-containing clusters')
    sns.heatmap(
        data=np.log10(normalize_by_clusters(df_interactions_all_clusters, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target clusters\nthat also contain columns targets'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[0]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[0])
    axs[0].set(title='Cluster count normalization')
    axs[0].tick_params(labelsize='xx-small')
    sns.heatmap(
        data=np.log10(normalize_by_pairwise(df_interactions_all_pairwise, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target colocalizations\ninvolving columns target'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[1]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[1])
    axs[1].set(title='Pairwise count normalization')
    axs[1].tick_params(labelsize='xx-small')
    fig.savefig(figs_out('interactions_all'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('interactions_all')))

## Ambiguous clusters

In [ ]:
if not all(map(
    os.path.exists,
    (csv_out('interactions_ambiguous_pairwise'),
     csv_out('interactions_ambiguous_clusters'),
     figs_out('interactions_ambiguous')))) or reprocess:

    mask_interacting_clusters = \
        (df_clusters['label'] == 'ambiguous') & \
        (df_clusters['chromatin count'] > 0) & \
        (df_clusters['mode oligo count'] > 1) & \
        (df_clusters['mode oligo count'] >= df_clusters['oligo count'] * 0.1)

    interactions_ambiguous_clusters = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_ambiguous_clusters = pd.DataFrame(interactions_ambiguous_clusters, index=labeled_targets, columns=labeled_targets)
    # save as DataFrame to preserve index and column labels
    df_interactions_ambiguous_clusters.to_csv(csv_out('interactions_ambiguous_clusters'))

    interactions_ambiguous_pairwise = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        downweighting='n_over_two',
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_ambiguous_pairwise = pd.DataFrame(interactions_ambiguous_pairwise, index=labeled_targets, columns=labeled_targets)
    df_interactions_ambiguous_pairwise.to_csv(csv_out('interactions_ambiguous_pairwise'))

    fig, axs = plt.subplots(1, 2, width_ratios=(1, 1), figsize=(10, 5), layout='constrained', gridspec_kw=dict(wspace=0.2))
    fig.suptitle('Colocalization in ambiguous clusters')
    sns.heatmap(
        data=np.log10(normalize_by_clusters(df_interactions_ambiguous_clusters, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target clusters\nthat also contain columns targets'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[0]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[0])
    axs[0].set(title='Cluster count normalization')
    axs[0].tick_params(labelsize='xx-small')
    sns.heatmap(
        data=np.log10(normalize_by_pairwise(df_interactions_ambiguous_pairwise, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target colocalizations\ninvolving columns target'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[1]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[1])
    axs[1].set(title='Pairwise count normalization')
    axs[1].tick_params(labelsize='xx-small')
    fig.savefig(figs_out('interactions_ambiguous'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('interactions_ambiguous')))

In [ ]:
if not all(map(
    os.path.exists,
    (csv_out('interactions_ambiguous_large_pairwise'),
     csv_out('interactions_ambiguous_large_clusters'),
     figs_out('interactions_ambiguous_large')))) or reprocess:

    mask_interacting_clusters = \
        (df_clusters['label'] == 'ambiguous') & \
        (df_clusters['chromatin count'] > 0) & \
        (df_clusters['mode oligo count'] > 1) & \
        (df_clusters['mode oligo count'] >= df_clusters['oligo count'] * 0.1)

    interactions_ambiguous_large_clusters = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_ambiguous_large_clusters = pd.DataFrame(interactions_ambiguous_large_clusters, index=labeled_targets, columns=labeled_targets)
    # save as DataFrame to preserve index and column labels
    df_interactions_ambiguous_large_clusters.to_csv(csv_out('interactions_ambiguous_large_clusters'))

    interactions_ambiguous_large_pairwise = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        downweighting='n_over_two',
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_ambiguous_large_pairwise = pd.DataFrame(interactions_ambiguous_large_pairwise, index=labeled_targets, columns=labeled_targets)
    df_interactions_ambiguous_large_pairwise.to_csv(csv_out('interactions_ambiguous_large_pairwise'))

    fig, axs = plt.subplots(1, 2, width_ratios=(1, 1), figsize=(10, 5), layout='constrained', gridspec_kw=dict(wspace=0.2))
    fig.suptitle('Colocalization in large ambiguous clusters')
    sns.heatmap(
        data=np.log10(normalize_by_clusters(df_interactions_ambiguous_large_clusters, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target clusters\nthat also contain columns targets'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[0]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[0])
    axs[0].set(title='Cluster count normalization')
    axs[0].tick_params(labelsize='xx-small')
    sns.heatmap(
        data=np.log10(normalize_by_pairwise(df_interactions_ambiguous_large_pairwise, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target colocalizations\ninvolving columns target'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[1]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[1])
    axs[1].set(title='Pairwise count normalization')
    axs[1].tick_params(labelsize='xx-small')
    fig.savefig(figs_out('interactions_ambiguous_large'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('interactions_ambiguous_large')))

## Filtered clusters

In [ ]:
if not all(map(
    os.path.exists,
    (csv_out('interactions_filtered_pairwise'),
     csv_out('interactions_filtered_clusters'),
     figs_out('interactions_filtered')))) or reprocess:

    mask_interacting_clusters = \
        (df_clusters['label'] == 'filtered') & \
        (df_clusters['chromatin count'] > 0) & \
        (df_clusters['mode oligo count'] > 1) & \
        (df_clusters['mode oligo count'] >= df_clusters['oligo count'] * 0.1)

    interactions_filtered_clusters = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_filtered_clusters = pd.DataFrame(interactions_filtered_clusters, index=labeled_targets, columns=labeled_targets)
    # save as DataFrame to preserve index and column labels
    df_interactions_filtered_clusters.to_csv(csv_out('interactions_filtered_clusters'))

    interactions_filtered_pairwise = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        downweighting='n_over_two',
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_filtered_pairwise = pd.DataFrame(interactions_filtered_pairwise, index=labeled_targets, columns=labeled_targets)
    df_interactions_filtered_pairwise.to_csv(csv_out('interactions_filtered_pairwise'))

    fig, axs = plt.subplots(1, 2, width_ratios=(1, 1), figsize=(10, 5), layout='constrained', gridspec_kw=dict(wspace=0.2))
    fig.suptitle('Colocalization in filtered clusters')
    sns.heatmap(
        data=np.log10(normalize_by_clusters(df_interactions_filtered_clusters, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target clusters\nthat also contain columns targets'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[0]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[0])
    axs[0].set(title='Cluster count normalization')
    axs[0].tick_params(labelsize='xx-small')
    sns.heatmap(
        data=np.log10(normalize_by_pairwise(df_interactions_filtered_pairwise, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target colocalizations\ninvolving columns target'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[1]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[1])
    axs[1].set(title='Pairwise count normalization')
    axs[1].tick_params(labelsize='xx-small')
    fig.savefig(figs_out('interactions_filtered'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('interactions_filtered')))

Interaction analysis of filtered clusters with high oligo counts

In [ ]:
high_oligo_count_thresh = 200

In [ ]:
if not all(map(
    os.path.exists,
    (csv_out('interactions_filtered_high_oligo_pairwise'),
     csv_out('interactions_filtered_high_oligo_clusters'),
     figs_out('interactions_filtered_high_oligo')))) or reprocess:

    mask_interacting_clusters = \
        (df_clusters['label'] == 'filtered') & \
        (df_clusters['chromatin count'] > 0) & \
        (df_clusters['oligo count'] >= high_oligo_count_thresh) & \
        (df_clusters['mode oligo count'] >= df_clusters['oligo count'] * 0.1)

    interactions_filtered_high_oligo_clusters = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_filtered_high_oligo_clusters = pd.DataFrame(interactions_filtered_high_oligo_clusters, index=labeled_targets, columns=labeled_targets)
    # save as DataFrame to preserve index and column labels
    df_interactions_filtered_high_oligo_clusters.to_csv(csv_out('interactions_filtered_high_oligo_clusters'))

    interactions_filtered_high_oligo_pairwise = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        downweighting='n_over_two',
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_filtered_high_oligo_pairwise = pd.DataFrame(interactions_filtered_high_oligo_pairwise, index=labeled_targets, columns=labeled_targets)
    df_interactions_filtered_high_oligo_pairwise.to_csv(csv_out('interactions_filtered_high_oligo_pairwise'))

    fig, axs = plt.subplots(1, 2, width_ratios=(1, 1), figsize=(10, 5), layout='constrained', gridspec_kw=dict(wspace=0.2))
    fig.suptitle('Colocalization in filtered_high_oligo clusters')
    sns.heatmap(
        data=np.log10(normalize_by_clusters(df_interactions_filtered_high_oligo_clusters, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target clusters\nthat also contain columns targets'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[0]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[0])
    axs[0].set(title='Cluster count normalization')
    axs[0].tick_params(labelsize='xx-small')
    sns.heatmap(
        data=np.log10(normalize_by_pairwise(df_interactions_filtered_high_oligo_pairwise, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target colocalizations\ninvolving columns target'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[1]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[1])
    axs[1].set(title='Pairwise count normalization')
    axs[1].tick_params(labelsize='xx-small')
    fig.savefig(figs_out('interactions_filtered_high_oligo'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('interactions_filtered_high_oligo')))

Interaction analysis of filtered clusters with low oligo counts

In [ ]:
low_oligo_counts_thresh = 30

In [ ]:
if not all(map(
    os.path.exists,
    (csv_out('interactions_filtered_low_oligo_pairwise'),
     csv_out('interactions_filtered_low_oligo_clusters'),
     figs_out('interactions_filtered_low_oligo')))) or reprocess:

    mask_interacting_clusters = \
        (df_clusters['label'] == 'filtered') & \
        (df_clusters['chromatin count'] > 0) & \
        (df_clusters['oligo count'] <= low_oligo_counts_thresh) & \
        (df_clusters['mode oligo count'] >= df_clusters['oligo count'] * 0.1)

    interactions_filtered_low_oligo_clusters = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_filtered_low_oligo_clusters = pd.DataFrame(interactions_filtered_low_oligo_clusters, index=labeled_targets, columns=labeled_targets)
    # save as DataFrame to preserve index and column labels
    df_interactions_filtered_low_oligo_clusters.to_csv(csv_out('interactions_filtered_low_oligo_clusters'))

    interactions_filtered_low_oligo_pairwise = label_counts_to_interaction_chunked(
        (label_distribution[barcode] for barcode in df_clusters.loc[mask_interacting_clusters].index),
        labeled_targets,
        downweighting='n_over_two',
        dtype_in=float,
        n_processes=n_processes)
    df_interactions_filtered_low_oligo_pairwise = pd.DataFrame(interactions_filtered_low_oligo_pairwise, index=labeled_targets, columns=labeled_targets)
    df_interactions_filtered_low_oligo_pairwise.to_csv(csv_out('interactions_filtered_low_oligo_pairwise'))

    fig, axs = plt.subplots(1, 2, width_ratios=(1, 1), figsize=(10, 5), layout='constrained', gridspec_kw=dict(wspace=0.2))
    fig.suptitle('Colocalization in filtered_low_oligo clusters')
    sns.heatmap(
        data=np.log10(normalize_by_clusters(df_interactions_filtered_low_oligo_clusters, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target clusters\nthat also contain columns targets'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[0]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[0])
    axs[0].set(title='Cluster count normalization')
    axs[0].tick_params(labelsize='xx-small')
    sns.heatmap(
        data=np.log10(normalize_by_pairwise(df_interactions_filtered_low_oligo_pairwise, pseudocount='min_over_ten')),
        cbar_kws=dict(label='log10 proportion of row target colocalizations\ninvolving columns target'),
        cbar_ax=mpl_toolkits.axes_grid1.make_axes_locatable(axs[1]).append_axes('right', size='5%', pad=0.05),
        square=True,
        xticklabels=True, yticklabels=True,
        ax=axs[1])
    axs[1].set(title='Pairwise count normalization')
    axs[1].tick_params(labelsize='xx-small')
    fig.savefig(figs_out('interactions_filtered_low_oligo'), bbox_inches='tight')
    fig.show()
else:
    display(Image(filename=figs_out('interactions_filtered_low_oligo')))